<a href="https://colab.research.google.com/github/persisbb/StockPredictor/blob/main/StockMarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.shape

(1374, 6)

In [ ]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,52.651001,53.799000,52.651001,53.660500,53.660500,31766000
2018-01-03,53.696499,54.805000,53.671501,54.576000,54.576000,31318000
2018-01-04,54.854500,55.203999,54.713001,54.787998,54.787998,26052000


Understanding Trends with in the Data

In [ ]:
# Sort the data points based on indexes just for confirmation
data.sort_index(inplace = True)

In [ ]:
# Remove any duplicate index
data = data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-14,123.099998,124.050003,121.449997,123.669998,123.669998,30592300
2023-06-15,123.139999,125.459999,122.400002,125.089996,125.089996,35233300
2023-06-16,125.930000,126.108002,123.879997,124.614998,124.614998,17370129


In [ ]:
# Check for missing values
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1374.000000,1374.000000,1374.000000,1374.000000,1374.000000,1.374000e+03
mean,87.228609,88.233285,86.259409,87.271138,87.271138,3.543448e+07
std,30.078694,30.388801,29.734435,30.058453,30.058453,1.592359e+07
min,49.216000,50.605999,48.882999,49.233501,49.233501,9.312000e+06
25%,59.298125,59.900002,58.660249,59.390374,59.390374,2.535450e+07
50%,77.498753,78.687500,76.202251,77.781250,77.781250,3.158600e+07
75%,112.345503,113.736748,111.178625,112.462376,112.462376,4.032200e+07
max,151.250000,151.546494,148.899002,149.838501,149.838501,1.331780e+08


In [ ]:
import plotly.graph_objects as go

# Check the trend in Closing Values
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900,
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900,
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pickle
from tqdm.notebook import tnrange

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
pip install sklearn


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Filter only required data
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,53.660500,31766000
2018-01-03,54.576000,31318000
2018-01-04,54.787998,26052000


In [ ]:
# Confirm the Testing Set length
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length):
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [ ]:
X , Y = CreateFeatures_and_Targets(data , 32)

NameError: name 'tnrange' is not defined

In [ ]:
# Check the shapes
X.shape , Y.shape

((724, 32, 2), (724,))

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
# Check Training Dataset Shape
Xtrain.shape , Ytrain.shape

((639, 32, 2), (639,))

In [ ]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((85, 32, 2), (85,))

In [ ]:
# Create a Scaler to Scale Vectors with Multiple Dimensions
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [ ]:
# Save your objects for future purposes
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [ ]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
639/639 [==============================] - 88s 130ms/step - loss: 0.0079 - val_loss: 0.0302
Epoch 2/10
639/639 [==============================] - 82s 128ms/step - loss: 0.0260 - val_loss: 0.0338
Epoch 3/10
639/639 [==============================] - 81s 127ms/step - loss: 0.0188 - val_loss: 0.0328
Epoch 4/10
639/639 [==============================] - 83s 130ms/step - loss: 0.0144 - val_loss: 0.0206
Epoch 5/10
639/639 [==============================] - 79s 123ms/step - loss: 0.0130 - val_loss: 0.0195
Epoch 6/10
639/639 [==============================] - 80s 126ms/step - loss: 0.0103 - val_loss: 0.0210
Epoch 7/10
639/639 [==============================] - 80s 125ms/step - loss: 0.0073 - val_loss: 0.0140
Epoch 8/10
639/639 [==============================] - 80s 126ms/step - loss: 0.0079 - val_loss: 0.0130
Epoch 9/10
639/639 [==============================] - 81s 127ms/step - loss: 0.0069 - val_loss: 0.0168
Epoch 10/10
639/639 [==============================] - 80s 126ms/step - l

In [ ]:
# Checking the model Structure
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (1, 32, 1024)             2109440   
_________________________________________________________________
lstm_1 (LSTM)                (1, 256)                  1311744   
_________________________________________________________________
dropout (Dropout)            (1, 256)                  0         
_________________________________________________________________
dense (Dense)                (1, 64)                   16448     
_________________________________________________________________
dropout_1 (Dropout)          (1, 64)                   0         
_________________________________________________________________
dense_1 (Dense)              (1, 32)                   2080      
_________________________________________________________________
dense_2 (Dense)              (1, 1)                    3

In [ ]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [ ]:
Predictions = model.predict(Xtest)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Creating Sample Test Dataframe
test_dataframe_dict = {'Actual' : list(Actual) , 'Predicted' : list(Predictions)}
test_df = pd.DataFrame.from_dict(test_dataframe_dict)

test_df.index = data.index[-test_length:]

In [ ]:
test_df.head()

,Actual,Predicted
Date,,
2020-09-01,1655.079956,1607.851196
2020-09-02,1717.390015,1614.339844
2020-09-03,1629.510010,1621.852539
2020-09-04,1581.209961,1623.274292
2020-09-08,1523.599976,1621.557983


In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = test_df.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = test_df.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [ ]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [ ]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [ ]:
Predictions = model.predict(Total_features)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()